# Bag Of Words (BOW)
O "Bag of Words" (BOW) é uma representação simplificada utilizada em processamento de linguagem natural (PLN) e recuperação de informações (RI) para representar textos como uma coleção desordenada de palavras. Ele ignora a ordem das palavras e captura a multiplicidade, ou seja, registra a contagem de cada palavra no texto.

Esse modelo é frequentemente usado em métodos de classificação de documentos, onde a frequência de cada palavra é usada como um recurso para treinar um classificador.

Para exemplificar o uso do BOW para modelar documentos textuais, vamos utilizar a biblioteca Scikit-Learn (Sklearn), mais especificamente o módulo [sklearn.feature_extraction.text](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text).

# CountVectorizer
Uma das formas mais simples de se utilizar o modelo Bag-Of-Words com o Scikit-Learn é com a classe [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer). Essa classe implementa um mecanismo para gerar representações BOW com a contagem de vezes que uma palavra aparece no texto.

A classe CountVectorizer implementa a interface de um BaseEstimator do sklearn, implementando uma API similar a de preditores/classificadores da biblioteca. O método .fit do vectorizer recebe como entrada textos e gera um dicionário de termos utilizados nesse texto.

Nesse dicionário é possível observar que as chaves/índices são as palavras presentes no texto, os valores associados representam indíces de colunas que estão associadas à essas palavras. No dicionário, ainda, é destacado que:
*   O termo 'top' está associado ao índice 4, enquanto o termo 'jogo' está associado ao índice 3;
*   Palavras com menos de 2 caracteres são descartadas, por terem baixa representatividade na configuração inicial da API Vectorizer;
*   Os termos são todos escritos em letras minúsculas.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

texts = ['Top Gear é um jogo de ação.']
vectorizer = CountVectorizer()
vectorizer.fit(texts)

print(vectorizer.vocabulary_)

{'top': 4, 'gear': 2, 'um': 5, 'jogo': 3, 'de': 1, 'ação': 0}



O método .transform recebe como entrada textos e gera uma representação de BOW considerando o dicionário de termos gerados na etapa anterior, pelo método .fit. No exemplo de código a seguir, observem que a API de Pandas e DataFrame só foi utilizada para melhorar a visibilidade do resultado.

In [ ]:
import pandas as pd

result = vectorizer.transform(['Top Gun foi um filme interessante de ação.'])

df = pd.DataFrame(result.toarray(), columns=vectorizer.get_feature_names_out())
print(df)

   ação  de  gear  jogo  top  um
0     1   1     0     0    1   1


No resultado, observem que cada linha possui o número de vezes que uma determinada palavra/termo apareceu na frase. Um ponto a se observar do exemplo anterior é que palavras que não estão presentes no dicionário gerado pelo método .fit são ignoradas na saída.

O dicionário de termos, apresentado anteriormente, indicava que o termo 'top' estava associado ao índice 4 e o termo 'jogo', ao índice 3. No resultado da transformação, esses associações podem ser observadas no DataFrame, dado que a coluna do termo 'top' é a coluna de número 4 e a coluna do termo 'jogo' é a coluna de número 3.

A API vectorizer permite que sejam dados como entrada múltiplos textos em vetores na definição dos dicionários de termos (pelo método .fit) e múltiplos textos também em vetores para gerarem a representação BOW (pelo método .transform).

In [ ]:
texts = ['Mineração de Dados foi interessante.',
         'Perceptron é o começo de Redes Neurais.']
vectorizer.fit(texts)
print(vectorizer.vocabulary_)

result = vectorizer.transform([
    'Aprendizado Profundo é parecido com Mineração?',
    'Perceptron é interessante mas não é legal.',
    'Outra disciplina interessante seria Cloud.',
    'Mineração de Dados usa conceitos de Mineração de Dados e Aprendizado de Máquina.',
])

print(pd.DataFrame(result.toarray(), columns=vectorizer.get_feature_names_out()))

{'mineração': 5, 'de': 2, 'dados': 1, 'foi': 3, 'interessante': 4, 'perceptron': 7, 'começo': 0, 'redes': 8, 'neurais': 6}
   começo  dados  de  foi  interessante  mineração  neurais  perceptron  redes
0       0      0   0    0             0          1        0           0      0
1       0      0   0    0             1          0        0           1      0
2       0      0   0    0             1          0        0           0      0
3       0      2   4    0             0          2        0           0      0


Nesse exemplo, foi definido um novo dicionário de termos, dada a chamada do método .fit. Foram utilizados termos presentes nos dois textos dados como entrada para o método (pela variável texts, ambos dentro de um vetor).

Em seguida, foram geradas representações BOW utilizando esse dicionário de termos, uma para cada texto dado como entrada para o método .transform.

Considerando, especificamente, a última frase dada como entrada para a API CountVectorizer, pode ser observado que os termos "dados" e "de" aparecem mais de uma vez no texto, com os números 2 e 4, respectivamente.

A classe CountVectorizer também permite configurações para que esses valores sejam contabilizados de forma binária. Nesse cenário, o valor 1 representaria situações em que o termo aparece no texto, sem contabilizar mais vezes caso esse termo tenha aparecido mais vezes.

In [ ]:
vectorizer = CountVectorizer(binary=True)

texts = ['Mineração de Dados foi interessante.',
         'Perceptron é o começo de Redes Neurais.']

vectorizer.fit(texts)
print(vectorizer.vocabulary_)

result = vectorizer.transform([
    'Mineração de Dados usa conceitos de Mineração de Dados e Aprendizado de Máquina.',
])

print(pd.DataFrame(result.toarray(), columns=vectorizer.get_feature_names_out()))

{'mineração': 5, 'de': 2, 'dados': 1, 'foi': 3, 'interessante': 4, 'perceptron': 7, 'começo': 0, 'redes': 8, 'neurais': 6}
   começo  dados  de  foi  interessante  mineração  neurais  perceptron  redes
0       0      1   1    0             0          1        0           0      0


# TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) é uma medida utilizada para avaliar a importância de uma palavra em um documento, em relação ao conjunto de documentos disponíveis. O TF-IDF é calculado a partir de duas medidas: a frequência do termo (TF) e a frequência inversa do documento (IDF). A frequência do termo (TF) é a porcentagem de ocorrências do termo em um documento, dividida pela porcentagem de ocorrências do termo em todos os documentos. A frequência inversa do documento (IDF) é a raiz natural do logaritmo do número total de documentos, dividido pelo número de documentos que contêm o termo.

O TF-IDF, diferentemente do CountVectorizer, é uma medida que combina a frequência de palavras em um documento com a frequência inversa das palavras em outros documentos. Isso ajuda a identificar palavras que são raras em um documento, mas comuns em outros, o que pode indicar que essas palavras são relevantes para a análise específica. O TF-IDF é mais complexo que o CountVectorizer, mas é mais útil para análises mais avançadas, como classificação de texto e busca de informações.

O Scikit-learn disponibiliza também uma classe que implementa a mesma interface Vectorizer do CountVectorizer. Nesse primeiro exemplo, vamos utilizar apenas o TF para representar o texto (sem o IDF, ainda).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=False)
texts = ['Mineração de Dados foi interessante.',
         'Perceptron é o começo de Redes Neurais.']

vectorizer.fit(texts)
print(vectorizer.vocabulary_)

result = vectorizer.transform([
    'Aprendizado Profundo é parecido com Mineração?',
    'Perceptron é interessante mas não é legal.',
    'Outra disciplina interessante seria Cloud.',
    'Mineração de Dados usa conceitos de Mineração de Dados e Aprendizado de Máquina.',
])

print(pd.DataFrame(result.toarray(), columns=vectorizer.get_feature_names_out()))

{'mineração': 5, 'de': 2, 'dados': 1, 'foi': 3, 'interessante': 4, 'perceptron': 7, 'começo': 0, 'redes': 8, 'neurais': 6}
   começo     dados        de  foi  interessante  mineração  neurais  \
0     0.0  0.000000  0.000000  0.0      0.000000   1.000000      0.0   
1     0.0  0.000000  0.000000  0.0      0.707107   0.000000      0.0   
2     0.0  0.000000  0.000000  0.0      1.000000   0.000000      0.0   
3     0.0  0.408248  0.816497  0.0      0.000000   0.408248      0.0   

   perceptron  redes  
0    0.000000    0.0  
1    0.707107    0.0  
2    0.000000    0.0  
3    0.000000    0.0  


Os resultados para cada palavra representam um valor de frequência que os termos aparecem nos documentos/linhas. Esses valores são calculados com base nos termos que aparecem mais vezes em cada valor de entrada e são normalizados considerando a estratégia padrão da classe [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

Nesse próximo exemplo, utilizaremos o cálculo do IDF.

In [ ]:
vectorizer = TfidfVectorizer()
texts = ['Mineração de Dados foi interessante.',
         'Perceptron é o começo de Redes Neurais.']

vectorizer.fit(texts)
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

result = vectorizer.transform([
    'Aprendizado Profundo é parecido com Mineração?',
    'Perceptron é interessante mas não é legal.',
    'Outra disciplina interessante seria Cloud.',
    'Mineração de Dados usa conceitos de Mineração de Dados e Aprendizado de Máquina.',
])

print(pd.DataFrame(result.toarray(), columns=vectorizer.get_feature_names_out()))

{'mineração': 5, 'de': 2, 'dados': 1, 'foi': 3, 'interessante': 4, 'perceptron': 7, 'começo': 0, 'redes': 8, 'neurais': 6}
[1.40546511 1.40546511 1.         1.40546511 1.40546511 1.40546511
 1.40546511 1.40546511 1.40546511]
   começo     dados        de  foi  interessante  mineração  neurais  \
0     0.0  0.000000  0.000000  0.0      0.000000   1.000000      0.0   
1     0.0  0.000000  0.000000  0.0      0.707107   0.000000      0.0   
2     0.0  0.000000  0.000000  0.0      1.000000   0.000000      0.0   
3     0.0  0.498446  0.709297  0.0      0.000000   0.498446      0.0   

   perceptron  redes  
0    0.000000    0.0  
1    0.707107    0.0  
2    0.000000    0.0  
3    0.000000    0.0  


Quando utilizamos o IDF para calcular os valores com a classe TfidfVectorizer, é definido um vetor, como atributo dos objetos vectorizer .idf_. Esse atributo contém os valores de IDF para cada termo do dicionário.

Nesse exemplo, considerando os textos utilizados como entrada para o método .fit, o único valor de IDF diferente foi observado para o termo "de". O termo "de" aparece nos dois documentos/linhas dados como entrada para o método .fit, por isso ele possui um valor de IDF menor. Nesse sentido, um IDF menor indica que o termo aparece mais vezes entre os documentos, por isso, tem um importância menor para representar os documentos.

É importante lembrar que os valores de IDF são calculados com base apenas nos documentos/textos utilizados na entrada do método .fit.

O resultado do método .transform também utiliza os valores de IDF para definir seus valores, como pode ser observado na saída do código anterior.

# Aplicações

O modelo de Bag-Of-Words possui diferentes tipos de aplicações. Como exemplos, são destacados seu uso nas áreas de:
*   Recuperação de Informação Textual;
*   Sistemas de Recomendação;
*   Classificação de Textos.

A Recuperação de Informação (RI) é o processo de identificar e localizar informações relevantes para um determinado problema ou questão. No contexto da Ciência da Informação, a RI pode ser definida como a operação pela qual se seleciona documentos de acordo com a demanda do usuário, ou como o fornecimento de elementos de informação documentária correspondentes a uma demanda definida pelo usuário.

Um Sistema de Recomendação, por outro lado, é um sistema que utiliza algoritmos e técnicas de análise de dados para ajudar os usuários a encontrarem informações, produtos ou serviços relevantes para eles. Esses sistemas são utilizados em diferentes áreas, como comércio eletrônico, busca de informações e plataformas de streaming, para oferecer sugestões personalizadas e melhorar a experiência do usuário.

O modelo Bag-Of-Words é  um dos modelos que pode ser utilizado para implementar estratégias de Recuperação de Informação e Sistemas de Recomendação. Nessas aplicações, o TF-IDF de documentos textuais ou palavras-chaves podem ser utilizados para construir um índice de busca. Esse índice pode ser posteriormente utilizado para que sejam realizadas buscas nos documentos com base em palavras-chave, utilizando como ordem de acesso aos documentos o valor de TF-IDF para cada termo.

Considere o seguinte exemplo de código:


In [ ]:
vectorizer = TfidfVectorizer()
texts = ['Mineração de Dados foi interessante.',
         'Perceptron é o começo de Redes Neurais.',
         'Aprendizado Profundo é parecido com Mineração?',
         'Perceptron é interessante mas não é legal.',
         'Outra disciplina interessante seria Cloud.',
         'Mineração de Dados usa conceitos de Mineração de Dados e Aprendizado de Máquina.',]

result = vectorizer.fit_transform(texts)

result_df = pd.DataFrame(result.toarray(), columns=vectorizer.get_feature_names_out())
print(result_df)

   aprendizado     cloud       com    começo  conceitos     dados        de  \
0     0.000000  0.000000  0.000000  0.000000   0.000000  0.464964  0.392555   
1     0.000000  0.000000  0.000000  0.490779   0.000000  0.000000  0.339772   
2     0.402446  0.000000  0.490779  0.000000   0.000000  0.000000  0.000000   
3     0.000000  0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
4     0.000000  0.472493  0.000000  0.000000   0.000000  0.000000  0.000000   
5     0.205337  0.000000  0.000000  0.000000   0.250407  0.410674  0.693439   

   disciplina       foi  interessante  ...   máquina   neurais       não  \
0    0.000000  0.567019      0.392555  ...  0.000000  0.000000  0.000000   
1    0.000000  0.000000      0.000000  ...  0.000000  0.490779  0.000000   
2    0.000000  0.000000      0.000000  ...  0.000000  0.000000  0.000000   
3    0.000000  0.000000      0.339772  ...  0.000000  0.000000  0.490779   
4    0.472493  0.000000      0.327113  ...  0.000000  0.000000  0.

Nesse código, nós construímos uma matriz contendo os valores de TF-IDF para todos os termos encontrados nos textos dados como entrada ao método .fit_transform. O método .fit_transform é um método que envolve as chamadas .fit e .transform em sequência. Ele pode ser utilizado quando se planeja chamar ambos os métodos com o mesmo parâmetro.

A tabela de valores apresentada pode ser utilizada, tanto para fins de busca por palavras-chave e sistemas de recomendação de conteúdo. Para ilustrar seu uso, vamos considerar que o usuário está realizando uma busca com o termo "perceptron" e "aprendizado profundo". Para determinar os documentos associados, poderíamos ordenar de forma decrescente utilizando os termos inseridos na busca.

In [ ]:
print(result_df[['perceptron', 'redes', 'neurais']]
      .sort_values(by=['redes', 'neurais', 'perceptron'], ascending=False))

   perceptron     redes   neurais
1    0.402446  0.490779  0.490779
3    0.402446  0.000000  0.000000
0    0.000000  0.000000  0.000000
2    0.000000  0.000000  0.000000
4    0.000000  0.000000  0.000000
5    0.000000  0.000000  0.000000


Nesse exemplo, o primeiro texto retornado seria o documento de índice 1 "Perceptron é o começo de Redes Neurais" e o segundo seria o índice 3 "Perceptron é interessante mas não é legal". No caso, essa seria a lista de conteúdos em ordem que poderia ser utilizada em um sistema de busca ou recomendação de conteúdo, com base em palavras-chave. Pode ser observado que os outros documentos não possuem valor de TF-IDF positivo, por isso poderiam ser excluídos.

Observem que o valor de TF-IDF dos termos "Redes" e "Neurais" também é maior que o valor do termo "Perceptron". Isso acontece pois o termo "Redes Neurais" apareceu para apenas um documento, sendo mais discriminativo que o termo "Perceptron" que apareceu em dois documentos. Por isso, esses termos foram utilizados com maior prioridade na ordenação dos documentos.

O cenário de Classificação de Textos vai ser explorado nas próximas aulas.